In [2]:
!pip install kaggle rasterio
from google.colab import files
files.upload()  # upload kaggle.json

# !mkdir -p ~/.kaggle
!cp kaggle\ \(1\).json ~/.kaggle/kaggle.json
# !cp kaggle*.json ~/.kaggle/kaggle.json
!kaggle competitions download -c 'ai-spacetech-hackathon'

!unzip -d dataset ai*.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/sar/SN6_AOI_2424.tif  
  inflating: dataset/train/sar/SN6_AOI_2425.tif  
  inflating: dataset/train/sar/SN6_AOI_2426.tif  
  inflating: dataset/train/sar/SN6_AOI_2427.tif  
  inflating: dataset/train/sar/SN6_AOI_2428.tif  
  inflating: dataset/train/sar/SN6_AOI_2429.tif  
  inflating: dataset/train/sar/SN6_AOI_243.tif  
  inflating: dataset/train/sar/SN6_AOI_2430.tif  
  inflating: dataset/train/sar/SN6_AOI_2431.tif  
  inflating: dataset/train/sar/SN6_AOI_2432.tif  
  inflating: dataset/train/sar/SN6_AOI_2433.tif  
  inflating: dataset/train/sar/SN6_AOI_2434.tif  
  inflating: dataset/train/sar/SN6_AOI_2435.tif  
  inflating: dataset/train/sar/SN6_AOI_2436.tif  
  inflating: dataset/train/sar/SN6_AOI_2439.tif  
  inflating: dataset/train/sar/SN6_AOI_244.tif  
  inflating: dataset/train/sar/SN6_AOI_2440.tif  
  inflating: dataset/train/sar/SN6_AOI_2441.tif  
  inflating: dataset/train/sar/SN6_AOI_2442.tif  
 

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
!nvidia-smi

Found GPU at: /device:GPU:0
Sun Mar 17 02:53:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              26W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+---------------------------------------

In [4]:
from pathlib import Path
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import cv2
import random
from tqdm import tqdm
import scipy.misc
import matplotlib.image
from sklearn.model_selection import train_test_split


dataset_dir = Path('dataset/train')

filenames = [f.name for f in Path('dataset/train/eo').iterdir()]

train_files, val_test_files, *_ = train_test_split(filenames, np.zeros((len(filenames), 1)), test_size=1 - 0.8, random_state=42)
val_files, test_files, *_ = train_test_split(val_test_files, np.zeros((len(val_test_files), 1)), test_size=0.5, random_state=42)

!rm -rf merged
!mkdir -p merged/{train,test,val}

for filenames, split in zip((train_files, val_files, test_files), ("train", "val", "test")):
  for i, file in tqdm(enumerate(filenames)):
    src_path = dataset_dir / "sar" / file
    dst_path = dataset_dir / "eo"  / file

    # Open the TIFF image using rasterio
    with rasterio.open(src_path) as src:
        # Read the image data
        img = src.read()

    img = img.squeeze()
    img /= 30
    img += 1
    img /= 2
    img *= 255

    resized_img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)
    img = cv2.cvtColor(resized_img/255,cv2.COLOR_GRAY2RGB)

    # Open the TIFF image using rasterio
    with rasterio.open(dst_path) as dst:
        # Read the image data
        img2 = dst.read([1,2,3])
        dst = cv2.normalize(img2, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    img2 = np.moveaxis(dst, 0, -1)
    img2 = img2[:256, :256, :]

    img3 = np.hstack((img2, img))

    matplotlib.image.imsave(f'merged/{split}/{file}.jpg', img3)


8733it [03:23, 42.84it/s]
1092it [00:25, 42.41it/s]
1092it [00:23, 45.95it/s]


In [5]:
!zip -r merged.zip merged

Streaming output truncated to the last 5000 lines.
  adding: merged/train/SN6_AOI_2932.tif.jpg (deflated 2%)
  adding: merged/train/SN6_AOI_293.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_1928.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_189.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_3486.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_1954.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_2596.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_12687.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_16462.tif.jpg (deflated 3%)
  adding: merged/train/SN6_AOI_7684.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_7217.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_8494.tif.jpg (deflated 2%)
  adding: merged/train/SN6_AOI_2016.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_7172.tif.jpg (deflated 2%)
  adding: merged/train/SN6_AOI_11714.tif.jpg (deflated 1%)
  adding: merged/train/SN6_AOI_15391.tif.jpg (deflated 1%)
  adding: merged/tr

In [6]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [8]:
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Pix2pix.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import tensorflow as tf

IMG_WIDTH = 256
IMG_HEIGHT = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE


def load(image_file):
  """Loads the image and generates input and target image.

  Args:
    image_file: .jpeg file

  Returns:
    Input image, target image
  """
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image


def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image


def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]


def normalize(input_image, real_image):
  # input_image = (input_image / 127.5) - 1
  # real_image = (real_image / 127.5) - 1

  return input_image, real_image


@tf.function
def random_jitter(input_image, real_image):
  """Random jittering.

  Resizes to 286 x 286 and then randomly crops to IMG_HEIGHT x IMG_WIDTH.

  Args:
    input_image: Input Image
    real_image: Real Image

  Returns:
    Input Image, real image
  """
  # resizing to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image


def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image


def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image


def create_dataset(path_to_train_images, path_to_test_images, buffer_size,
                   batch_size):
  """Creates a tf.data Dataset.

  Args:
    path_to_train_images: Path to train images folder.
    path_to_test_images: Path to test images folder.
    buffer_size: Shuffle buffer size.
    batch_size: Batch size

  Returns:
    train dataset, test dataset
  """
  train_dataset = tf.data.Dataset.list_files(path_to_train_images)
  train_dataset = train_dataset.shuffle(buffer_size)
  train_dataset = train_dataset.map(
      load_image_train, num_parallel_calls=AUTOTUNE)
  train_dataset = train_dataset.batch(batch_size)

  test_dataset = tf.data.Dataset.list_files(path_to_test_images)
  test_dataset = test_dataset.map(
      load_image_test, num_parallel_calls=AUTOTUNE)
  test_dataset = test_dataset.batch(batch_size)

  return train_dataset, test_dataset


class InstanceNormalization(tf.keras.layers.Layer):
  """Instance Normalization Layer (https://arxiv.org/abs/1607.08022)."""

  def __init__(self, epsilon=1e-5):
    super(InstanceNormalization, self).__init__()
    self.epsilon = epsilon

  def build(self, input_shape):
    self.scale = self.add_weight(
        name='scale',
        shape=input_shape[-1:],
        initializer=tf.random_normal_initializer(1., 0.02),
        trainable=True)

    self.offset = self.add_weight(
        name='offset',
        shape=input_shape[-1:],
        initializer='zeros',
        trainable=True)

  def call(self, x):
    mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
    inv = tf.math.rsqrt(variance + self.epsilon)
    normalized = (x - mean) * inv
    return self.scale * normalized + self.offset


def downsample(filters, size, norm_type='batchnorm', apply_norm=True):
  """Downsamples an input.

  Conv2D => Batchnorm => LeakyRelu

  Args:
    filters: number of filters
    size: filter size
    norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
    apply_norm: If True, adds the batchnorm layer

  Returns:
    Downsample Sequential Model
  """
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_norm:
    if norm_type.lower() == 'batchnorm':
      result.add(tf.keras.layers.BatchNormalization())
    elif norm_type.lower() == 'instancenorm':
      result.add(InstanceNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result


def upsample(filters, size, norm_type='batchnorm', apply_dropout=False):
  """Upsamples an input.

  Conv2DTranspose => Batchnorm => Dropout => Relu

  Args:
    filters: number of filters
    size: filter size
    norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
    apply_dropout: If True, adds the dropout layer

  Returns:
    Upsample Sequential Model
  """

  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

  if norm_type.lower() == 'batchnorm':
    result.add(tf.keras.layers.BatchNormalization())
  elif norm_type.lower() == 'instancenorm':
    result.add(InstanceNormalization())

  if apply_dropout:
    result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result


def unet_generator(output_channels, norm_type='batchnorm'):
  """Modified u-net generator model (https://arxiv.org/abs/1611.07004).

  Args:
    output_channels: Output channels
    norm_type: Type of normalization. Either 'batchnorm' or 'instancenorm'.

  Returns:
    Generator model
  """

  down_stack = [
      downsample(64, 4, norm_type, apply_norm=False),  # (bs, 128, 128, 64)
      downsample(128, 4, norm_type),  # (bs, 64, 64, 128)
      downsample(256, 4, norm_type),  # (bs, 32, 32, 256)
      downsample(512, 4, norm_type),  # (bs, 16, 16, 512)
      downsample(512, 4, norm_type),  # (bs, 8, 8, 512)
      downsample(512, 4, norm_type),  # (bs, 4, 4, 512)
      downsample(512, 4, norm_type),  # (bs, 2, 2, 512)
      downsample(512, 4, norm_type),  # (bs, 1, 1, 512)
  ]

  up_stack = [
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 2, 2, 1024)
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 4, 4, 1024)
      upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 8, 8, 1024)
      upsample(512, 4, norm_type),  # (bs, 16, 16, 1024)
      upsample(256, 4, norm_type),  # (bs, 32, 32, 512)
      upsample(128, 4, norm_type),  # (bs, 64, 64, 256)
      upsample(64, 4, norm_type),  # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 4, strides=2,
      padding='same', kernel_initializer=initializer,
      activation='tanh')  # (bs, 256, 256, 3)

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[None, None, 3])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


def discriminator(norm_type='batchnorm', target=True):
  """PatchGan discriminator model (https://arxiv.org/abs/1611.07004).

  Args:
    norm_type: Type of normalization. Either 'batchnorm' or 'instancenorm'.
    target: Bool, indicating whether target image is an input or not.

  Returns:
    Discriminator model
  """

  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[None, None, 3], name='input_image')
  x = inp

  if target:
    tar = tf.keras.layers.Input(shape=[None, None, 3], name='target_image')
    x = tf.keras.layers.concatenate([inp, tar])  # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, norm_type, False)(x)  # (bs, 128, 128, 64)
  down2 = downsample(128, 4, norm_type)(down1)  # (bs, 64, 64, 128)
  down3 = downsample(256, 4, norm_type)(down2)  # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(
      512, 4, strides=1, kernel_initializer=initializer,
      use_bias=False)(zero_pad1)  # (bs, 31, 31, 512)

  if norm_type.lower() == 'batchnorm':
    norm1 = tf.keras.layers.BatchNormalization()(conv)
  elif norm_type.lower() == 'instancenorm':
    norm1 = InstanceNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(norm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(
      1, 4, strides=1,
      kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 30, 1)

  if target:
    return tf.keras.Model(inputs=[inp, tar], outputs=last)
  else:
    return tf.keras.Model(inputs=inp, outputs=last)


def get_checkpoint_prefix():
  checkpoint_dir = './training_checkpoints'
  checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

  return checkpoint_prefix


class Pix2pix(object):
  """Pix2pix class.

  Args:
    epochs: Number of epochs.
    enable_function: If true, train step is decorated with tf.function.
    buffer_size: Shuffle buffer size..
    batch_size: Batch size.
  """

  def __init__(self, epochs, enable_function):
    self.epochs = epochs
    self.enable_function = enable_function
    self.lambda_value = 100
    self.loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    # self.generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    # self.discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    self.generator = unet_generator(output_channels=3)
    self.discriminator = discriminator()
    self.checkpoint = tf.train.Checkpoint(
        generator_optimizer=self.generator_optimizer,
        discriminator_optimizer=self.discriminator_optimizer,
        generator=self.generator,
        discriminator=self.discriminator)

  def discriminator_loss(self, disc_real_output, disc_generated_output):
    real_loss = self.loss_object(
        tf.ones_like(disc_real_output), disc_real_output)

    generated_loss = self.loss_object(tf.zeros_like(
        disc_generated_output), disc_generated_output)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

  def generator_loss(self, disc_generated_output, gen_output, target):
    gan_loss = self.loss_object(tf.ones_like(
        disc_generated_output), disc_generated_output)

    # mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = gan_loss + (self.lambda_value * l1_loss)
    return total_gen_loss, l1_loss

  def train_step(self, input_image, target_image):
    """One train step over the generator and discriminator model.

    Args:
      input_image: Input Image.
      target_image: Target image.

    Returns:
      generator loss, discriminator loss.
    """
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      gen_output = self.generator(input_image, training=True)

      disc_real_output = self.discriminator(
          [input_image, target_image], training=True)
      disc_generated_output = self.discriminator(
          [input_image, gen_output], training=True)

      gen_loss, l1_loss = self.generator_loss(
          disc_generated_output, gen_output, target_image)
      disc_loss = self.discriminator_loss(
          disc_real_output, disc_generated_output)

    generator_gradients = gen_tape.gradient(
        gen_loss, self.generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(
        disc_loss, self.discriminator.trainable_variables)

    self.generator_optimizer.apply_gradients(zip(
        generator_gradients, self.generator.trainable_variables))
    self.discriminator_optimizer.apply_gradients(zip(
        discriminator_gradients, self.discriminator.trainable_variables))

    return gen_loss, disc_loss, l1_loss

  def train(self, dataset, checkpoint_pr):
    """Train the GAN for x number of epochs.

    Args:
      dataset: train dataset.
      checkpoint_pr: prefix in which the checkpoints are stored.

    Returns:
      Time for each epoch.
    """
    time_list = []
    if self.enable_function:
      self.train_step = tf.function(self.train_step)

    for epoch in range(self.epochs):
      start_time = time.time()
      for input_image, target_image in dataset:
        gen_loss, disc_loss, l1_loss = self.train_step(input_image, target_image)

      wall_time_sec = time.time() - start_time
      time_list.append(wall_time_sec)

      # saving (checkpoint) the model every 20 epochs
      if (epoch + 1) % 1 == 0:
        self.checkpoint.save(file_prefix=checkpoint_pr)

      template = 'Epoch {}, Generator loss {}, L1 Loss {}, Discriminator Loss {}'
      print (template.format(epoch, gen_loss, l1_loss, disc_loss))

    return time_list


def run_main():
  # kwargs = {'epochs': FLAGS.epochs, 'enable_function': FLAGS.enable_function,
  #           'path': FLAGS.path, 'buffer_size': FLAGS.buffer_size,
  #           'batch_size': FLAGS.batch_size}

  kwargs = {'epochs': 20, 'enable_function': True,
            'path': 'merged', 'buffer_size': len(train_files),
            'batch_size': 16
  }

  return main(**kwargs)


def main(epochs, enable_function, path, buffer_size, batch_size):
  path_to_folder = path

  pix2pix_object = Pix2pix(epochs, enable_function)

  train_dataset, _ = create_dataset(
      os.path.join(path_to_folder, 'train/*.jpg'),
      os.path.join(path_to_folder, 'val/*.jpg'),
      buffer_size, batch_size)
  checkpoint_pr = get_checkpoint_prefix()
  print ('Training ...')
  return pix2pix_object.train(train_dataset, checkpoint_pr)


with tf.device('/device:GPU:0'):
  pix2pix = run_main()

Training ...
Epoch 0, Generator loss 5115.86279296875, L1 Loss 51.08191680908203, Discriminator Loss 0.0007524488610215485
Epoch 1, Generator loss 3535.710205078125, L1 Loss 35.270790100097656, Discriminator Loss 0.0003540643956512213
Epoch 2, Generator loss 3912.055419921875, L1 Loss 39.02717208862305, Discriminator Loss 0.0002148529456462711
Epoch 3, Generator loss 3809.987548828125, L1 Loss 37.99967575073242, Discriminator Loss 7.098217611201108e-05
Epoch 4, Generator loss 3464.815185546875, L1 Loss 34.54450225830078, Discriminator Loss 5.872799374628812e-05
Epoch 5, Generator loss 4508.91015625, L1 Loss 44.98127365112305, Discriminator Loss 3.264135375502519e-05
Epoch 6, Generator loss 3174.5810546875, L1 Loss 31.6347599029541, Discriminator Loss 3.176958853146061e-05
Epoch 7, Generator loss 3268.731689453125, L1 Loss 32.571895599365234, Discriminator Loss 1.670021447353065e-05
Epoch 8, Generator loss 3894.40087890625, L1 Loss 38.824974060058594, Discriminator Loss 1.26081431517377

##### Inference

In [9]:
import pickle

with open('p2p.pkl', 'wb') as f:
    pickle.dump(pix2pix, f)


In [34]:
!ls /content/training_checkpoints

checkpoint		     ckpt-16.index		  ckpt-3.index
ckpt-10.data-00000-of-00001  ckpt-17.data-00000-of-00001  ckpt-4.data-00000-of-00001
ckpt-10.index		     ckpt-17.index		  ckpt-4.index
ckpt-11.data-00000-of-00001  ckpt-18.data-00000-of-00001  ckpt-5.data-00000-of-00001
ckpt-11.index		     ckpt-18.index		  ckpt-5.index
ckpt-12.data-00000-of-00001  ckpt-19.data-00000-of-00001  ckpt-6.data-00000-of-00001
ckpt-12.index		     ckpt-19.index		  ckpt-6.index
ckpt-13.data-00000-of-00001  ckpt-1.data-00000-of-00001   ckpt-7.data-00000-of-00001
ckpt-13.index		     ckpt-1.index		  ckpt-7.index
ckpt-14.data-00000-of-00001  ckpt-20.data-00000-of-00001  ckpt-8.data-00000-of-00001
ckpt-14.index		     ckpt-20.index		  ckpt-8.index
ckpt-15.data-00000-of-00001  ckpt-2.data-00000-of-00001   ckpt-9.data-00000-of-00001
ckpt-15.index		     ckpt-2.index		  ckpt-9.index
ckpt-16.data-00000-of-00001  ckpt-3.data-00000-of-00001


In [38]:
generator = unet_generator(3)
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [39]:
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
checkpoint.restore(tf.train.latest_checkpoint('training_checkpoints'))

#Got stuck in loading the model to perform inference

ValueError: `Checkpoint` was expecting discriminator to be a trackable object (an object derived from `Trackable`), got <function discriminator at 0x7fc34e545090>. If you believe this object should be trackable (i.e. it is part of the TensorFlow Python API and manages state), please open an issue.

In [29]:
# from pathlib import Path

# filenames = list(Path('dataset/test/sar').iterdir())

# img = rasterio.open(filenames[0]).read()
# img = np.moveaxis(img, 0, -1)


# from IPython import get_ipython
# from IPython.core.magics.namespace import NamespaceMagics
# _nms = NamespaceMagics()
# _Jupyter = get_ipython()
# _nms.shell = _Jupyter.kernel.shell

# _nms.who_ls()

['AUTOTUNE',
 'IMG_HEIGHT',
 'IMG_WIDTH',
 'Image',
 'InstanceNormalization',
 'NamespaceMagics',
 'Path',
 'Pix2pix',
 'absolute_import',
 'cm',
 'create_dataset',
 'cv2',
 'dataset_dir',
 'device_name',
 'discriminator',
 'division',
 'downsample',
 'dst',
 'dst_path',
 'f',
 'file',
 'filenames',
 'files',
 'get_checkpoint_prefix',
 'get_ipython',
 'i',
 'img',
 'img2',
 'img3',
 'load',
 'load_image_test',
 'load_image_train',
 'main',
 'matplotlib',
 'normalize',
 'np',
 'os',
 'pickle',
 'pix2pix',
 'plt',
 'print_function',
 'random',
 'random_crop',
 'random_jitter',
 'rasterio',
 'reshape_as_image',
 'reshape_as_raster',
 'resize',
 'resized_img',
 'run_main',
 'scipy',
 'split',
 'src',
 'src_path',
 'test_files',
 'tf',
 'time',
 'tqdm',
 'train_files',
 'train_test_split',
 'unet_generator',
 'upsample',
 'val_files',
 'val_test_files']